In [1]:
# !pip install transformers datasets
!pip install ast
import os
os.environ['TRANSFORMERS_CACHE'] = '/p/project/training2411/kumar/'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

  Using cached AST-0.0.2.tar.gz (19 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-73ue37d6/ast_2e66089896cb4a0f91e576d1914ebe6c/setup.py", line 6, in <module>
          README = codecs.open(os.path.join(here, 'AST/README'), encoding='utf8').read()
        File "/p/project/training2411/kumar/miniconda/envs/peft/lib/python3.9/codecs.py", line 905, in open
          file = builtins.open(filename, mode, buffering)
      FileNotFoundError: [Errno 2] No such file or directory: '/tmp/pip-install-73ue37d6/ast_2e66089896cb4a0f91e576d1914ebe6c/AST/README'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-gen

In [2]:
eval

<function eval(source, globals=None, locals=None, /)>

In [3]:
import pandas as pd
from datasets import Dataset, ClassLabel
from transformers import RobertaTokenizerFast
from transformers import DataCollatorForTokenClassification

# Load the dataset
df = pd.read_csv('ner_new.csv', converters={'tokens': eval, 'ner_tags': eval})
data = Dataset.from_pandas(df)

/p/project/training2411/kumar/miniconda/envs/peft/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [9]:
data[4]

{'tokens': ['Satellite',
  'images',
  'of',
  'burn',
  'scars',
  'in',
  'Montana',
  'from',
  'August',
  '14',
  ',',
  '2023',
  '.'],
 'ner_tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'LOCATION',
  'O',
  'DATE',
  'DATE',
  'O',
  'DATE',
  'O']}

In [10]:
# Define unique labels
unique_labels = ['O', 'DATE', 'LOCATION']  # add all your labels here
label_dict = {label: i for i, label in enumerate(unique_labels)}

# Update the dataset with encoded labels
def encode_labels(examples):
    try:
        return {'ner_tags': [label_dict[label] for label in examples['ner_tags']]}
    except:
        return None

data = data.map(encode_labels)

def convert_list(input_list):
    output_list = input_list
    for i in range(2, len(output_list) - 1):
        if output_list[i - 2] == 1 and output_list[i - 1] == 1 and output_list[i] == 0 and output_list[i + 1] == 1:
            output_list[i] = 1
            break
    return output_list



def consolidate_labels(dataset):
    dataset["ner_tags"] = convert_list(dataset["ner_tags"])
        
    return dataset

data = data.map(consolidate_labels)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [11]:
for i, item in enumerate(data):
    print(item)
        

{'tokens': ['February', '8', ',', '2023', ':', 'Provide', 'satellite', 'imagery', 'of', 'flooding', 'in', 'Miami', '.'], 'ner_tags': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0]}
{'tokens': ['Burn', 'scars', 'in', 'Oregon', ',', 'noted', 'on', 'April', '17', ',', '2023', '.'], 'ner_tags': [0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0]}
{'tokens': ['July', '25', ',', '2023', ':', 'Identify', 'crop', 'types', 'in', 'Northern', 'India', '.'], 'ner_tags': [1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0]}
{'tokens': ['Flooding', 'in', 'central', 'China', 'on', 'June', '30', ',', '2023', '.'], 'ner_tags': [0, 0, 0, 2, 0, 1, 1, 1, 1, 0]}
{'tokens': ['Satellite', 'images', 'of', 'burn', 'scars', 'in', 'Montana', 'from', 'August', '14', ',', '2023', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 1, 0]}
{'tokens': ['December', '5', ',', '2023', ':', 'Show', 'crop', 'types', 'in', 'Chile', '.'], 'ner_tags': [1, 1, 1, 1, 0, 0, 0, 0, 2, 0]}
{'tokens': ['March', '22', ',', '2023', ',', 'highlight', 'flooding', 'events', 

In [12]:
# Load tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', add_prefix_space=True, use_cache=True, cache_dir="/p/project/training2411/kumar")

# Function to tokenize and align labels
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        print(label, word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            try:
                if word_idx is None:
                    label_ids.append(-100)
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(label[word_idx] if label_all_tokens else -100)
                previous_word_idx = word_idx  # Update previous_word_idx inside the loop
            except Exception as e:  # Use Exception instead of error
                print(f"Error in {word_idx}")
                print(e)
                print(f"{label}")
                continue
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = data.map(tokenize_and_align_labels, batched=True, batch_size=128)
data_collator = DataCollatorForTokenClassification(tokenizer)

/p/project/training2411/kumar/miniconda/envs/peft/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0] [None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, None]
[0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0] [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 11, None]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0] [None, 0, 1, 2, 3, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, None]
[0, 0, 0, 2, 0, 1, 1, 1, 1, 0] [None, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, None]
[0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 1, 0] [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 12, None]
[1, 1, 1, 1, 0, 0, 0, 0, 2, 0] [None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, None]
Error in 10
list index out of range
[1, 1, 1, 1, 0, 0, 0, 0, 2, 0]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 0] [None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, None]
[0, 1, 1, 1, 1, 0, 0, 0, 0, 2, 0, 2, 0] [None, 0, 1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, None]
Error in 13
list index out of range
[0, 1, 1, 1, 1, 0, 0, 0, 0, 2, 0, 2, 0]
[0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0] [None, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 11, None]
[1, 1, 1, 1, 0, 0, 0, 2, 

In [13]:
tokenized_datasets[1]

{'tokens': ['Burn',
  'scars',
  'in',
  'Oregon',
  ',',
  'noted',
  'on',
  'April',
  '17',
  ',',
  '2023',
  '.'],
 'ner_tags': [0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0],
 'input_ids': [0,
  7960,
  26172,
  11,
  4316,
  2156,
  1581,
  15,
  587,
  601,
  2156,
  291,
  1922,
  479,
  2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 1, 0, -100]}

In [14]:
from transformers import RobertaForTokenClassification, Trainer, TrainingArguments

model = RobertaForTokenClassification.from_pretrained(
    'roberta-base',
    use_cache=True,
    cache_dir="/p/project/training2411/kumar",
    num_labels=len(unique_labels) # This should match your total number of NER tags
)


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
    learning_rate=2e-4,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Assuming you have a train split
    eval_dataset=tokenized_datasets,  # Assuming you have a train split
    data_collator = data_collator
)

trainer.train()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,No log,0.051451
2,No log,0.231514
3,No log,0.031607
4,No log,0.042400
5,No log,0.037117
6,No log,0.022357
7,No log,0.015271
8,No log,0.010495
9,No log,0.006297
10,No log,0.005953


TrainOutput(global_step=130, training_loss=0.0453031246478741, metrics={'train_runtime': 11.4361, 'train_samples_per_second': 86.568, 'train_steps_per_second': 11.367, 'total_flos': 8461600380360.0, 'train_loss': 0.0453031246478741, 'epoch': 10.0})

In [19]:
model.save_pretrained('./finetuned_roberta_ner')
tokenizer.save_pretrained('./finetuned_roberta_ner')


('./finetuned_roberta_ner/tokenizer_config.json',
 './finetuned_roberta_ner/special_tokens_map.json',
 './finetuned_roberta_ner/vocab.json',
 './finetuned_roberta_ner/merges.txt',
 './finetuned_roberta_ner/added_tokens.json',
 './finetuned_roberta_ner/tokenizer.json')

In [21]:
text = "Crop types in Spain durin 23 April, 2023."
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
print(inputs)
for k, v in inputs.items():
    print(k, v)

{'input_ids': tensor([[    0,   230,  6884,  3505,    11,  2809, 17373,   179,   883,   587,
             6,   291,  1922,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
input_ids tensor([[    0,   230,  6884,  3505,    11,  2809, 17373,   179,   883,   587,
             6,   291,  1922,     4,     2]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [22]:
from transformers import pipeline

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)



In [70]:
text = "Crop types in Spain as of yesterday."
ner_classes = ner_pipeline(text)

In [80]:
ner_classes

[{'entity': 'LABEL_0',
  'score': 0.99953103,
  'index': 1,
  'word': 'ĠC',
  'start': 0,
  'end': 1},
 {'entity': 'LABEL_0',
  'score': 0.9995999,
  'index': 2,
  'word': 'rop',
  'start': 1,
  'end': 4},
 {'entity': 'LABEL_0',
  'score': 0.9996145,
  'index': 3,
  'word': 'Ġtypes',
  'start': 5,
  'end': 10},
 {'entity': 'LABEL_0',
  'score': 0.9985916,
  'index': 4,
  'word': 'Ġin',
  'start': 11,
  'end': 13},
 {'entity': 'LABEL_2',
  'score': 0.9999062,
  'index': 5,
  'word': 'ĠSpain',
  'start': 14,
  'end': 19},
 {'entity': 'LABEL_0',
  'score': 0.9990376,
  'index': 6,
  'word': 'Ġas',
  'start': 20,
  'end': 22},
 {'entity': 'LABEL_0',
  'score': 0.9990075,
  'index': 7,
  'word': 'Ġof',
  'start': 23,
  'end': 25},
 {'entity': 'LABEL_0',
  'score': 0.9964805,
  'index': 8,
  'word': 'Ġyesterday',
  'start': 26,
  'end': 35},
 {'entity': 'LABEL_0',
  'score': 0.8945812,
  'index': 9,
  'word': '.',
  'start': 35,
  'end': 36}]

In [81]:
def ner_to_dict(ner_result):
    result = dict()
    for item in ner_result:
        if item['entity'] in ["LABEL_1", "LABEL_2"]:
            if item['entity'] in result.keys():
                result[item['entity']] += item['word']
            else:
                result[item['entity']] = ""
                result[item['entity']] += item['word']
    old_keys = list(result.keys())
    
    for key in old_keys:
        new_key = unique_labels[int(key[-1])]
        result[new_key] = result[key]
        del result[key]
    
    for key, val in result.items():
        result[key] = val.replace('Ġ', ' ')[1:]

    return result

print(ner_to_dict(ner_classes))

{'LOCATION': 'Spain'}


In [82]:
result

{'LOCATION': 'Spain', 'DATE': '23 April, 2023'}